In [51]:
from llama_cpp import Llama
llm = Llama(
      model_path="/Users/ashishmadkaikar/code/llama.cpp/models/ggml-model-q5_k.gguff",
      #n_gpu_layers=-1, # Uncomment to use GPU acceleration
      seed=1337, # Uncomment to set a specific seed
      n_ctx=10000, # Uncomment to increase the context window
)


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/ashishmadkaikar/code/llama.cpp/models/mistral-7b-v0.1.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:

In [ ]:
import json
from chromadb.config import Settings
from chromadb import Chroma
from chromadb.utils import Query, Text

# Load JSON data
with open('/mnt/data/topics/topics.json', 'r') as file:
    data = json.load(file)

# Initialize ChromaDB
chroma = Chroma(Settings(chroma_path='chroma_db'))

# Create a new collection
collection = chroma.create_collection('manuals')

# Helper function to flatten the nested JSON data and insert into ChromaDB
def flatten_and_insert(data, parent_key='', collection=None):
    for k, v in data.items():
        key = f'{parent_key}.{k}' if parent_key else k
        if isinstance(v, dict):
            flatten_and_insert(v, key, collection)
        else:
            doc = {
                'key': key,
                'content': v
            }
            collection.insert(doc)

# Flatten and insert the JSON data
flatten_and_insert(data, collection=collection)

# Define the search query
search_query = "Introduction, Installation, Features, Specifications, Contact"

# Perform the search
query = Query(
    query=search_query,
    k=10,  # limit results to 10
    fields=['key', 'content']
)
results = collection.search(query)

# Print the search results
for i, result in enumerate(results['hits']):
    print(f"Result {i+1}:")
    print(f"Key: {result['key']}")
    print(f"Content: {result['content']}\n")


In [56]:
output = llm(
      "Q: Explain the word espunged A: ", # Prompt
      max_tokens=2000, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      echo=True # Echo the prompt back in the output
) # Generate a completion, can also call create_completion
print(output)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     729.95 ms
llama_print_timings:      sample time =       1.97 ms /    26 runs   (    0.08 ms per token, 13218.10 tokens per second)
llama_print_timings: prompt eval time =     486.66 ms /    10 tokens (   48.67 ms per token,    20.55 tokens per second)
llama_print_timings:        eval time =    1506.21 ms /    25 runs   (   60.25 ms per token,    16.60 tokens per second)
llama_print_timings:       total time =    2001.57 ms /    35 tokens


{'id': 'cmpl-108c2eaf-b067-4a09-b558-1a7c78184fa4', 'object': 'text_completion', 'created': 1719924049, 'model': '/Users/ashishmadkaikar/code/llama.cpp/models/mistral-7b-v0.1.Q5_K_S.gguf', 'choices': [{'text': 'Q: Explain the word espunged A:  It is a fancy way of saying removed.  The definition from my dictionary is: To pull out; to draw forth.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 13, 'completion_tokens': 26, 'total_tokens': 39}}


In [59]:
from llama_cpp import Llama
from PIL import Image
import torch
#from torchvision import transforms
from transformers import BlipProcessor, BlipForConditionalGeneration

# Initialize Llama model
llm = Llama(
    model_path="/Users/ashishmadkaikar/code/llama.cpp/models/llava/ggml-model-q5_k.gguf",
    # n_gpu_layers=-1, # Uncomment to use GPU acceleration
    seed=1337, # Uncomment to set a specific seed
    n_ctx=10000, # Uncomment to increase the context window
)

# Load the BLIP model and processor for image captioning
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Load and preprocess the image
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return image

def generate_caption(image):
    inputs = processor(images=image, return_tensors="pt")
    outputs = model.generate(**inputs)
    caption = processor.decode(outputs[0], skip_special_tokens=True)
    return caption

# Ask question about the image
def ask_question(image_path, question):
    image = load_image(image_path)
    caption = generate_caption(image)
    print(f"Caption: {caption}")
    
    prompt = f"Image description: {caption}\nQ: {question} A:"
    output = llm(
        prompt,
        max_tokens=2000,
        stop=["Q:", "\n"],
        echo=True
    )
    return output




llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/ashishmadkaikar/code/llama.cpp/models/llava/ggml-model-q5_k.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 ll

: 

In [2]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:8000/v1", api_key="llama.cpp")
response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://user-images.githubusercontent.com/1991296/230134379-7181e485-c521-4d23-a0d6-f7b3b61ba524.png",
                    },
                },
                {"type": "text", "text": "What does the image say. Format your response as a json object with a single 'text' key."},
            ],
        }
    ],
    response_format={ 
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {
                "text": {
                    "type": "string"
                }
            }
        }
    }
)
import json
print(json.loads(response.choices[0].message.content))

APIConnectionError: Connection error.

In [4]:
from flask import Flask, request, jsonify
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer

#app = Flask(__name__)

# Load the CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

#@app.route('/api/multimodal', methods=['POST'])
def multimodal_endpoint():
    # Get the image and question from the request
    image_file = request.files['image']
    question = request.form['question']
    
    # Open and preprocess the image
    image = Image.open(image_file)
    inputs = processor(text=question, images=image, return_tensors="pt", padding=True)
    
    # Get the model's response
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1)  # we can use this as the model's "confidence" for the answer

    # For simplicity, return the highest probability label
    answer = tokenizer.decode(torch.argmax(probs, dim=1))
    
    return jsonify({'answer': answer})

# if __name__ == '__main__':
#     app.run(port=8080)


/Users/ashishmadkaikar/miniforge3/envs/bulletin-generator/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
